[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, collections, IPython, mistune, doctest, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    from pidgin import tangle, shell, formatter

In [3]:
    class DocTestGrammar:
        doctest = doctest.DocTestParser._EXAMPLE_RE
        
    class DoctestLexer:
        def parse_doctest(self, m): self.tokens.append({'type': 'doctest', 'text': m.string[slice(*m.span())]})

In [4]:
    class GraphvizGrammar:
        graphviz = re.compile("^(di){0,1}graph\s*\{(\{(\{(\{(\{(\{.*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}", re.DOTALL)
    class GraphvizLexer:
        def parse_graphviz(self, m): 
            self.tokens.append({'type': 'graphviz', 'text': m.string[slice(*m.span())]})

In [5]:
    class InlineLexer(mistune.InlineLexer):
        def output_reflink(self, m):
            key = mistune._keyify(m.group(2) or m.group(1))
            if key in self.renderer.user_ns:
                value = self.renderer.user_ns[key]
                if isinstance(value, str) and value.startswith(('http:', 'https:')):
                    self.links[key] = dict(title="", link=str(value))
                else:
                    ret = self.links[key] = dict(title=str(value), link="")
                return self._process_link(m, ret['link'], ret['title'])
            return super().output_reflink(m)
        
        def output_emphasis(self, m):
            object = m.string[slice(*m.span())]
            if object not in self.renderer.user_ns:
                self.renderer.user_ns[object] = object
            text = m.group(2) or m.group(1)
            text = self.output(text)
            return self.renderer.emphasis(text)

In [6]:
    class BlockLexer(GraphvizLexer, DoctestLexer, mistune.BlockLexer): 
        class grammar_class(GraphvizGrammar, DocTestGrammar, mistune.BlockGrammar): ...
        
        def __init__(BlockLexer, rules=None, **kwargs): super().__init__(rules, **kwargs)
        
        def parse(self, text: str, rules=None) -> t.List[t.Dict]: 
            from pidgin import transform_cell
            return super().parse(transform_cell.strip_blank_lines(text))
        
        def parse_block_code(self, m=None): 
            self.tokens[-1]['lang'] = super().parse_block_code(m) or ''
            
        def parse_block_quote(self, m):
            object = m.string[slice(*m.span())]
            if object not in self.shell.user_ns: self.shell.user_ns[object] = object
            super().parse_block_quote(m)
            
    for attr in "doctest graphviz".split():
        for key in "default_rules footnote_rules list_rules".split():[
            setattr(BlockLexer, key, list(BlockLexer.default_rules)), 
            getattr(BlockLexer, key).insert(getattr(BlockLexer, key).index('block_quote'), attr)]

In [7]:
    class Markdown(mistune.Renderer, mistune.Markdown, shell.Shell):
        run = traitlets.Bool(default_value=True)
        def __init__(Markdown, *args, **kwargs):
            shell.Shell.__init__(Markdown, *args, **kwargs)
            mistune.Renderer.__init__(Markdown)
            mistune.Markdown.__init__(Markdown, renderer=Markdown, block=BlockLexer, inline=InlineLexer)
            Markdown.block.shell = Markdown
            Markdown._codespan_buffer = []
            Markdown._block_code_buffer = []

        def __enter__(Markdown): 
            Markdown.run = False
            return Markdown
        
        def __exit__(Markdown, *e): Markdown.run = True
            
        def codespan(Markdown, text, lang=None):
            Markdown._codespan_buffer.append(text)
            if not text.endswith('//'):
                Markdown.run_literal('\n'.join(Markdown._codespan_buffer))
                Markdown._codespan_buffer = []
            return super().codespan(text)
        
        def block_code(Markdown, text, lang=None):
            if lang is None: 
                Markdown._block_code_buffer.append(text)
                if not text.endswith('//'):
                    Markdown.run_literal('\n'.join(Markdown._block_code_buffer))
                    Markdown._block_code_buffer = []
            lang = lang or 'python'
            import pygments
            return pygments.highlight(text, pygments.lexers.get_lexer_by_name(lang, stripall=True), pygments.formatters.html.HtmlFormatter(noclasses=True))
        
        def run_literal(Markdown, str):
            return Markdown.run and super().run_cell(textwrap.indent(str, ' '*4), silent=True)
        
        def output_doctest(self): return self.renderer.doctest(self.token['text'])
        
        def output_graphviz(self): return self.renderer.graphviz(self.token['text'])
        
        def output_reflink(self, m):
            key = _keyify(m.group(2) or m.group(1))
            if key not in self.links:
                return None
            ret = self.links[key]
            return self._process_link(m, ret['link'], ret['title'])
                
        def doctest(self, text): return super().block_code(text)
        def graphviz(self, text): 
            return __import__('graphviz').Source(text)._repr_svg_()